In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [1]:
import gymnasium as gym
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy
import tqdm
import os

2025-01-05 19:41:52.357821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736102512.371733    5269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736102512.375825    5269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 19:41:52.389590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# prepare output folders of model snapshots and logs for tensorboard
models_dir, logs_dir = "models/lunarlander/PPO", "logs/lunarlander"

In [7]:
# recreate env and load a formerly saved snapshot and render animations from it
env = gym.make("LunarLander-v3", render_mode="rgb_array")
env.reset()

model = PPO.load(f"{models_dir}/330000.0", env=env)

vec_env = model.get_env()
obs = vec_env.reset()

episodes = 5
total_reward_episode = 0

for episode in range(episodes):
    # VecEnv resets automatically but one could optionally reset it here
    # obs = vec_env.reset()
    done = False    
    total_reward_episode = 0
    while not done:
        action, _state = model.predict(obs, deterministic=True)
        obs, reward, done, info = vec_env.step(action)
        vec_env.render("human")
        total_reward_episode += reward

    print(f"Total reward in episode {episode + 1} was {total_reward_episode}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Total reward in episode 1 was [200.04858]
Total reward in episode 2 was [203.38184]
Total reward in episode 3 was [-26.626305]
Total reward in episode 4 was [232.75142]
Total reward in episode 5 was [-40.87177]
